## docu

### status code 200

- 200 is ok for get request. 
- 201 is created with post request

### code 300

- 302 is redirect

### code 400

- 401 is unauthorized. to access data without the correct credentials
- 404 is page not found
- 409 is already exists. to create an existing entity

## code

In [1]:
import httpx
import re
import json

In [2]:
import lxml.html

In [3]:
URLS = [
        'https://yandex.ru/',
        'https://weather.data-analyst.praktikum-services.ru/v1/forecast',
        'https://fakestoreapi.com/products',
        'https://store.steampowered.com/explore/new/'
        ]
PARAM = {'page': '4'}
params1 = {
        'city' : 'Moscow',
        'hours' : True
    }

In [17]:
res = httpx.get(url=URLS[2], params=params1) 

In [4]:
res = httpx.get(url=URLS[4-1])

In [5]:
f'code {res.status_code}'

'code 200'

In [ ]:
res.json()

In [ ]:
for prod in res.json():
    print(prod['price'])

In [ ]:
json.loads(res.text)

In [5]:
html = lxml.html.fromstring(res.content)

In [6]:
new_releases = html.xpath('//div[@id="tab_newreleases_content"]')[0]
titles = new_releases.xpath('.//div[@class="tab_item_name"]/text()')
prices = new_releases.xpath('.//div[@class="discount_final_price"]/text()')

output = []

for info in zip(titles, prices):
    games = {}
    games['title'] = info[0]
    games['price'] = info[1]
    output.append(games)

output[:3]

[{'title': 'Atari 50: The Anniversary Celebration', 'price': '$18.49'},
 {'title': 'Lords and Villeins', 'price': '$16.49'},
 {'title': 'Among Us VR', 'price': '$6.29'}]

## POST testing

In [12]:
payload = {
    'title': 'new product',
    'price': 13.5,
    'description': 'test description',
    'image': '',
    'category': 'electronic'
}

In [15]:
res_post = httpx.post(url=URLS[2], data=payload)

In [18]:
res_post.json()

{'id': 21,
 'title': 'new product',
 'price': '13.5',
 'description': 'test description',
 'image': '',
 'category': 'electronic'}

## delete testing

In [19]:
res_del = httpx.delete('https://fakestoreapi.com/products/21')

In [21]:
res_del.json()

## JS

In [ ]:
fetch('https://fakestoreapi.com/products',
  ).then(res =>{
    if (res.ok){
    return res.json()
}
}).then(response=>{
    console.log(response)
}).catch(err => console.log(err))

In [ ]:
fetch('https://fakestoreapi.com/products',
  ).then(res =>{
    if (res.ok){
    return res.json()
}
}).then(response=>{
    response.map(data =>{
        console.log(data.price)
    })
    // console.log(response)
}).catch(err => console.log(err))

In [ ]:
// Specify the payload 
let payload = {
    title: 'new product',
    price: 13.5,
    description: 'test description',
    image: '',
    category: 'electronic'
}

fetch('https://fakestoreapi.com/products',
{
    method: "Post",
    headers:{
        'Content-Type': 'application/json'  
    },
    body: JSON.stringify(payload) //convert the payload to JSON 
}
  ).then(res =>{
      if (res.ok){
    console.log(res.status)
    return res.json()
}
}).then(response => {
    console.log(response)
}).catch(err => console.log(err))

In [ ]:
fetch('https://fakestoreapi.com/products/19',
{
    method: "Put",
    headers:{
        'Content-Type': 'application/json'  
    },
    body: JSON.stringify(payload) //convert the payload into JSON
}
  ).then(res =>{
      if (res.ok){
    console.log(res.status)
    return res.json()
}
}).then(response => {
    console.log(response)
}).catch(err => console.log(err))

In [ ]:
fetch('https://fakestoreapi.com/products/19',
{
    method: "Delete",
    headers:{
        'Content-Type': 'application/json'  
    }
}
  ).then(res =>{
      if (res.ok){
    console.log(res.status)
    return res.json()
}
}).then(response => {
    console.log(response)
}).catch(err => console.log(err))